In [10]:
import pandas as pd
import cv2
import os
import numpy as np

In [11]:
new_df=[]
for cur,_,filename in os.walk("/kaggle/input/visual-anomaly-detection-thermostatic-valves/training"):
    for file in filename:
        data=os.path.join(cur,file)
        cv=cv2.imread(data)
        if cv is not None:
            cv3=cv2.resize(cv,(256,256))
            new_df.append(cv3)

In [12]:
new_df_array=np.array(new_df)

In [13]:
new_df_array.shape

(121, 256, 256, 3)

# # Deep learning autoencoder

In [14]:
from keras.layers import Dense,Flatten,Conv2D,MaxPooling2D,TimeDistributed,BatchNormalization,Input,LeakyReLU,Dropout,Conv2DTranspose
from keras.models import Model
from keras.initializers import HeNormal
from keras.regularizers import l2
from keras.callbacks import EarlyStopping

In [15]:
early_stopping=EarlyStopping(
monitor='val_loss',
patience=5,
verbose=1)

In [16]:
inputs=Input(shape=(256, 256, 3))
# e=Conv2D(256,(3,3),activation=None,padding='same',kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(inputs)
# e=LeakyReLU(0.8)(e)
# e=BatchNormalization()(e)
# e=Dropout(0.5)(e)
e=Conv2D(128,(3,3),activation=None,padding='same',kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(inputs)
e=LeakyReLU(0.8)(e)
e=BatchNormalization()(e)
e=Dropout(0.5)(e)
e=Conv2D(64,(3,3),activation="relu",padding='same',kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(e)
# e=LeakyReLU(0.8)(e)
e=BatchNormalization()(e)
# e=Dropout(0.5)(e)

e=Conv2DTranspose(32,(3,3),activation="relu",padding='same',kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(e)
# e=LeakyReLU(0.8)(e)
e=BatchNormalization()(e)
e=Dropout(0.5)(e)

d=Conv2DTranspose(64,(3,3),activation=None,padding='same',kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(e)
d=LeakyReLU(0.8)(d)
d=BatchNormalization()(d)
d=Dropout(0.5)(d)
d=Conv2DTranspose(128,(3,3),activation=None,padding='same',kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(d)
d=LeakyReLU(0.8)(d)
d=BatchNormalization()(d)
# d=Dropout(0.5)(d)
# d=Conv2DTranspose(256,(3,3),activation="relu",padding='same',kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(d)
# # e=LeakyReLU(0.8)(e)
# d=BatchNormalization()(d)
# e=Dropout(0.5)(e)

outputs=Conv2D(3,(1,1),activation="sigmoid")(d)

model=Model(inputs,outputs)
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=["accuracy"])

In [17]:
model.fit(new_df_array,new_df_array,epochs=32,batch_size=32,validation_split=0.2,callbacks=[early_stopping])

Epoch 1/32
3/3 ━━━━━━━━━━━━━━━━━━━━ 21s 4s/step - accuracy: 0.3449 - loss: -14.1845 - val_accuracy: 0.1049 - val_loss: 2167.1289
Epoch 2/32
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 343ms/step - accuracy: 0.3694 - loss: -145.9051 - val_accuracy: 0.1005 - val_loss: -75720.2500
Epoch 3/32
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 342ms/step - accuracy: 0.3130 - loss: -237.9070 - val_accuracy: 0.1005 - val_loss: -238837.5938
Epoch 4/32
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 340ms/step - accuracy: 0.2779 - loss: -282.2725 - val_accuracy: 0.1006 - val_loss: -396574.8125
Epoch 5/32
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 341ms/step - accuracy: 0.2650 - loss: -309.8122 - val_accuracy: 0.1006 - val_loss: -477205.2500
Epoch 6/32
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 340ms/step - accuracy: 0.2690 - loss: -332.2323 - val_accuracy: 0.1007 - val_loss: -479635.8125
Epoch 7/32
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 341ms/step - accuracy: 0.2729 - loss: -356.6887 - val_accuracy: 0.1008 - val_loss: -437707.8125
Epoch 8/32
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 343ms/step - accuracy: 

In [24]:
prediction=model.predict(new_df_array)
reconstruction_error=np.mean(np.square(new_df_array-prediction),axis=(1,2,3))
threshold=np.mean(reconstruction_error)+2*np.std(reconstruction_error)
labels=np.where(reconstruction_error>threshold,0,1)
accuracy=np.mean(labels==0)*100
print(f"accuracy:{accuracy:.2f}%")

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
accuracy:4.13%


In [25]:
labels

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1])

In [26]:
true_labels = np.ones_like(labels)
accuracy = np.mean(labels == true_labels) * 100

print(f"Accuracy: {accuracy:.2f}%")

Accuracy: 95.87%
